In [15]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU

# Coleta de Dados (2 anos)
btc = yf.Ticker("BTC-USD")
btc_data = btc.history(period="2y", interval="1h")  # Coleta de dados por hora

# Salvar dados em um CSV
data = pd.read_csv("btc_data.csv", encoding='utf-8', parse_dates=['Datetime'], index_col='Datetime')

# Coluna 'Close' para previsão
data = data[["Close"]]

# Normalizar os dados
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Cria as sequências para o modelo GRU
sequence_length = 60
X, y = [], []

for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i - sequence_length : i, 0])
    y.append(scaled_data[i, 0])

X, y = np.array(X), np.array(y)

# Redimensionar X
X = np.reshape(X, (X.shape[0], X.shape[1], 1))
# Obter as datas correspondentes para y
y_dates = data.index[sequence_length:]

# Divisão dos Dados
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
y_dates_train, y_dates_test = y_dates[:split], y_dates[split:]

# Construção do Modelo GRU
model = Sequential()
model.add(GRU(units=100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(GRU(units=100))
model.add(Dense(1))

model.compile(optimizer="adam", loss="mean_squared_error")

# Treinamento do modelo
model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=1)

# Fazer Previsões
predicted_prices = model.predict(X_test)
predicted_prices = scaler.inverse_transform(predicted_prices.reshape(-1, 1))

# Ajustar o tamanho de y_test para corresponder ao tamanho de predicted_prices
real_prices = y_test[:len(predicted_prices)]
real_prices = scaler.inverse_transform(real_prices.reshape(-1, 1))

# Previsão das próximas 4 dias (96 horas)
last_sequence = X_test[-1]
predictions = []

# Gerar previsões para as próximas 96 horas
for _ in range(96):
    pred = model.predict(last_sequence.reshape(1, sequence_length, 1))
    predictions.append(pred[0, 0])
    last_sequence = np.append(last_sequence[1:], pred)[-sequence_length:]
    last_sequence = last_sequence.reshape(sequence_length, 1)

predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

# Obter as datas para as próximas 96 horas
last_date = y_dates_test[-1]
future_dates = pd.date_range(start=last_date + pd.Timedelta(hours=1), periods=96, freq='H')

# Cálculo das Métricas do Modelo
mse = mean_squared_error(real_prices, predicted_prices)
rmse = np.sqrt(mse)
mae = mean_absolute_error(real_prices, predicted_prices)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")

# Identificação dos Melhores Horários para Comprar e Vender
sell_hour_index = np.argmax(predictions)
buy_hour_index = np.argmin(predictions)

sell_hour = future_dates[sell_hour_index]
buy_hour = future_dates[buy_hour_index]

print(f"Melhor horário para vender Bitcoin: {sell_hour}")
print(f"Melhor horário para comprar Bitcoin: {buy_hour}")

# Gráfico mostrando a previsão para as próximas 96 horas comparada aos dados passados
plt.figure(figsize=(14, 7))

# Dados históricos de preços
plt.plot(data.index, data['Close'], label="Dados Históricos", color="black")

# Previsões futuras
plt.plot(future_dates, predictions, label="Previsão 96 Horas Futuras", color="blue")

# Melhor horário para vender
plt.scatter(sell_hour, predictions[sell_hour_index], color="red", label="Melhor Horário para Vender", marker="v", s=100)

# Melhor horário para comprar
plt.scatter(buy_hour, predictions[buy_hour_index], color="green", label="Melhor Horário para Comprar", marker="^", s=100)

plt.title("Previsão de Preço do Bitcoin e Melhores Horários para Compra e Venda")
plt.xlabel("Data e Hora")
plt.ylabel("Preço Previsto (USD)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Gráfico de comparação entre preços reais e previsões
plt.figure(figsize=(14, 7))
plt.plot(y_dates_test[:len(predicted_prices)], real_prices, color="black", label="Preços Reais")
plt.plot(y_dates_test[:len(predicted_prices)], predicted_prices, color="blue", label="Previsões")
plt.title("Comparação entre Preços Reais e Previsões")
plt.xlabel("Data e Hora")
plt.ylabel("Preço (USD)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Erros de previsão
errors = real_prices - predicted_prices

plt.figure(figsize=(14, 7))
plt.plot(y_dates_test[:len(errors)], errors, color="red", label="Erros de Previsão")
plt.title("Erros de Previsão (Preço Real - Preço Previsto)")
plt.xlabel("Data e Hora")
plt.ylabel("Erro (USD)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Gráfico das previsões para as próximas 96 horas
plt.figure(figsize=(14, 7))
plt.plot(y_dates_test[-96:], real_prices[-96:], label="Últimos Preços Reais", color="black")
plt.plot(future_dates, predictions, label="Previsões Próximas 96 Horas", color="blue")

plt.title("Previsões de Preço para as Próximas 96 Horas")
plt.xlabel("Data e Hora")
plt.ylabel("Preço (USD)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Distribuição dos erros de previsão
plt.figure(figsize=(10, 6))
plt.hist(errors, bins=50, color="purple", alpha=0.7)
plt.title("Distribuição dos Erros de Previsão")
plt.xlabel("Erro (USD)")
plt.ylabel("Frequência")
plt.show()

# Salvar o modelo treinado
model.save('model_gru_btc.h5')


ValueError: Missing column provided to 'parse_dates': 'Date'